# Track length per contract region (and year)

## Read input data

In [20]:
import pandas as pd # type: ignore

# Step 1: Load the Excel file 
excel_file_path = "../Python matching/raw_data/Kontraktsområde, kontrakt, bandel.xlsx"
contract_df = pd.read_excel(excel_file_path, sheet_name="Kontraktsområden 2011-2023")
length_df = pd.read_excel(excel_file_path, sheet_name="Bandelar & driftpl 2011-2023")

In [21]:
# rename column Kontraktsområdesnamn [kan behöva justeras] to Kontraktsområdesnamn
contract_df.rename(columns={'Kontraktsområdesnamn [kan behöva justeras]': "Kontraktsområdesnamn"}, inplace=True)
# rename column Kontraktsnummer \n(Diarienummer) to Kontraktsnummer
contract_df.rename(columns={"Kontraktsnummer \n(Diarienummer)": "Kontraktsnummer"}, inplace=True)

In [22]:
print(contract_df.columns)
print(length_df.columns)

Index(['År', 'Kontraktsområdesnamn', 'Fiktivt områdesnummer',
       'Kontraktsnummer', 'Starttid', 'Sluttid', 'Bandel', 'Del av bandel',
       'Övergång till nytt område', 'Övergång från gammalt område',
       'Kommentar', 'Antagande'],
      dtype='object')
Index(['år', 'bandel_fr', 'plats_fr', 'bandel_ti', 'plats_ti', 'km_fr_min',
       'km_ti_max'],
      dtype='object')


In [23]:
excel_file_path = "../Python matching/raw_data/BIS-data 2024-01-09 - Bandel, plats och förbindelselinje, alla spår.xlsx"
bis_df = pd.read_excel(excel_file_path)
bis_df.columns

Index(['Banlangd', 'BdlNr', 'Bandel', 'Plats_sign', 'Plats', 'Forbind',
       'Spår_huvud_sido', 'UNE', 'Spårnummer'],
      dtype='object')

## Processing

In length_df, there are some rows where bandel_ti is missing, we use the existing mapping (between plats_fr and bandel_fr/km_fr_min, removing duplicates) to find missing bandel_ti and km_ti_max based on plats_ti.

In [24]:
# Create a mapping between (år, plats_fr) and km_fr_min
km_mapping = length_df[['år', 'plats_fr', 'km_fr_min']].drop_duplicates().set_index(['år', 'plats_fr'])['km_fr_min'].to_dict()
# Create a mapping between (år, plats_fr) and bandel_fr
bdl_mapping = length_df[['år', 'plats_fr', 'bandel_fr']].drop_duplicates().set_index(['år', 'plats_fr'])['bandel_fr'].to_dict()

# Fill in the missing bandel_ti and km_ti_max values based on plats_ti and år
length_df['bandel_ti'] = length_df.apply(lambda row: bdl_mapping.get((row['år'], row['plats_ti']), row['bandel_ti']), axis=1)
length_df['km_ti_max'] = length_df.apply(lambda row: km_mapping.get((row['år'], row['plats_ti']), row['km_ti_max']), axis=1)

In [25]:
# remove rows with missing km_ti_max
length_df = length_df[length_df['km_ti_max'].notnull()]

In [5]:
# # remove rows from contract_df where Övergång till nytt område is 1
# print(len(contract_df))
# contract_df = contract_df[contract_df["Övergång till nytt område"] != 1]
# print(len(contract_df))

In [26]:
# if duplicates (same 'År', 'Bandel') are found in contract_df, keep the one with Övergång från gammalt område = 1
# sort by 'Övergång från gammalt område' and drop duplicates
print(len(contract_df))
contract_df = contract_df.sort_values('Övergång från gammalt område').drop_duplicates(subset=['År', 'Bandel'], keep='first')
# reset the order of the rows
contract_df = contract_df.sort_index()
print(len(contract_df))

3360
3116


## Track section length (km_fr_ti)

For each year, we first add columns (Kontraktsområdesnamn_fr and Kontraktsområdesnamn_ti) to length_df based on (bandel_fr and bandel_til) using contract_df. We have the matching between Kontraktsområdesnamn and Bandel for a given År.

In [27]:
# Merge length_df with contract_df to get Kontraktsområdesnamn_fr
length_df_processed = length_df.merge(contract_df[['År', 'Bandel', 'Kontraktsområdesnamn']], 
                            left_on=['år', 'bandel_fr'], 
                            right_on=['År', 'Bandel'], 
                            how='left')
length_df_processed.rename(columns={'Kontraktsområdesnamn': 'Kontraktsområdesnamn_fr'}, inplace=True)
length_df_processed.drop(columns=['År', 'Bandel'], inplace=True)

# Merge length_df_processed with contract_df to get Kontraktsområdesnamn_ti
length_df_processed = length_df_processed.merge(contract_df[['År', 'Bandel', 'Kontraktsområdesnamn']], 
                            left_on=['år', 'bandel_ti'], 
                            right_on=['År', 'Bandel'], 
                            how='left')
length_df_processed.rename(columns={'Kontraktsområdesnamn': 'Kontraktsområdesnamn_ti'}, inplace=True)
length_df_processed.drop(columns=['År', 'Bandel'], inplace=True)

In [28]:
# remove rows where Kontraktsområdesnamn_fr and Kontraktsområdesnamn_ti is NaN
print(len(length_df_processed))
length_df_processed = length_df_processed[(length_df_processed['Kontraktsområdesnamn_fr'].notna()) & (length_df_processed['Kontraktsområdesnamn_ti'].notna())]
print(len(length_df_processed))

37911
34413


In [29]:
# check if bandel_fr and bandel_ti are the same
print(length_df_processed["Kontraktsområdesnamn_fr"].equals(length_df_processed["Kontraktsområdesnamn_ti"]))
length_df_processed[length_df_processed["Kontraktsområdesnamn_fr"] != length_df_processed["Kontraktsområdesnamn_ti"]]

False


,år,bandel_fr,plats_fr,bandel_ti,plats_ti,km_fr_min,km_ti_max,Kontraktsområdesnamn_fr,Kontraktsområdesnamn_ti
117,2011,120,Bdn,137.0,Bud,1147.52994,1152.514020,Luleå-Gällivare-Koskullskulle,Haparandabanan
118,2011,120,Bud,137.0,Bud,1147.52999,1152.514020,Luleå-Gällivare-Koskullskulle,Haparandabanan
157,2011,124,Ht,120.0,Bds,1136.23002,1141.700020,Holmsund-Boden Södra inklusive tvärbanor,Luleå-Gällivare-Koskullskulle
206,2011,129,Dgm,146.0,Vns,847.93802,859.133020,Långsele-Vännäs och Mellansel-Örnsköldsvik,Holmsund-Boden Södra inklusive tvärbanor
207,2011,130,Fsm,153.0,Fsm,0.45069,0.749724,Långsele-Vännäs och Mellansel-Örnsköldsvik,Hällnäs-Storuman och Forsmo-Hoting
...,...,...,...,...,...,...,...,...,...
37724,2023,909,Hm,941.0,Hm,0.35800,1.250000,"Södra stambanan 2, del 1+2",Blekinge kustbana och kust till kustbana
37777,2023,925,Lma,901.0,Al,289.92300,613.553000,Västkustbanan Syd,Malmö sydöstra Skåne
37795,2023,931,Bjm,941.0,Hm,56.67600,1.250000,Västkustbanan Syd,Blekinge kustbana och kust till kustbana
37825,2023,935,Tp,912.0,E,34.09900,584.867000,Västkustbanan Syd,"Södra stambanan 2, del 1+2"


In [30]:
# remove rows where Kontraktsområdesnamn_fr and Kontraktsområdesnamn_ti are different
print(len(length_df_processed))
length_df_processed = length_df_processed[length_df_processed["Kontraktsområdesnamn_fr"] == length_df_processed["Kontraktsområdesnamn_ti"]]
print(len(length_df_processed))
# remove column Kontraktsområdesnamn_ti and rename Kontraktsområdesnamn_fr to Kontraktsområdesnamn
length_df_processed.drop(columns=['Kontraktsområdesnamn_ti'], inplace=True)
length_df_processed.rename(columns={'Kontraktsområdesnamn_fr': 'Kontraktsområdesnamn'}, inplace=True)

34413
28024


Calculate the accumulated lengths between (km_fr and km_ti) from length_df and add it to contract_df.

In [32]:
# Step 1: Calculate the total length for each bandel and year
length_df_processed['km_fr_ti'] = abs(length_df_processed['km_ti_max'] - length_df_processed['km_fr_min']) # convert to km
length_per_contract_year = length_df_processed.groupby(['år', 'Kontraktsområdesnamn'])['km_fr_ti'].sum().reset_index()

In [13]:
# Add column km_fr_ti to contract_df_merged by merging on year and contract region
contract_df_merged = contract_df.merge(length_per_contract_year, left_on=['År', 'Kontraktsområdesnamn'], right_on=['år', 'Kontraktsområdesnamn'], how='left')
contract_df_merged.drop(columns=['år'], inplace=True)